# CIFAR-10 with TensorFlow
Source: [Solaris Lab](http://solarisailab.com)

Assignment 1에서 다뤘던 CIFAR-10 클래시피케이션을 텐서플로로 구현해 봅시다.

이 작업은 트레이닝이 오래 걸립니다. GCP를 쓰고 있다면 GPU를 추가해 봅시다.

- `sudo /opt/deeplearning/install-driver.sh`으로 추가할 수 있습니다. 
- 추가한 후 `sudo reboot`으로 재부팅을 해야 합니다.
- `pip3 install --upgrade tensorflow-gpu`으로 텐서플로 GPU 지원을 설치합니다.
- 연결된 장치 리스트는 아래 커맨드로 조회할 수 있습니다.

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

일단 데이터를 로드합니다. CIFAR-10의 경우에도 데이터셋을 쉽게 받아올 수 있습니다.

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets.cifar10 import load_data

다음 배치를 읽어오기 위한 유틸리티 함수를 정의합니다.

In [ ]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

CNN 모델을 정의합니다. CONV1 -> POOL1 -> CONV2 -> CONV3 -> CONV4 -> CONV5 -> FC1 -> FC2.

* `tf.truncated_normal`: 정규분포에서 랜덤한 값으로 배열 생성
* `tf.nn.relu`: ReLU
* `tf.nn.conv2d`: 2D CONV
    * `padding='SAME'`: CONV 레이어를 거치고도 원래 사이즈가 유지되도록 패딩합니다.
* `tf.reshape`: NumPy의 그것과 같음
* `tf.nn.dropout`: 특정 확률로 데이터 드롭아웃

In [ ]:
def build_CNN_classifier(x):
    # 입력 이미지
    x_image = x

    # CONV
    W_conv1 = tf.Variable(
        tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
    b_conv1 = tf.Variable(
        tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.relu(
        tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

    # POOL
    h_pool1 = tf.nn.max_pool(
        h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

    # CONV
    W_conv2 = tf.Variable(
        tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
    b_conv2 = tf.Variable(
        tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(
        tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

    # POOL
    h_pool2 = tf.nn.max_pool(
        h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

    # CONV
    W_conv3 = tf.Variable(
        tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
    b_conv3 = tf.Variable(
        tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.relu(
        tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

    # CONV
    W_conv4 = tf.Variable(
        tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    b_conv4 = tf.Variable(
        tf.constant(0.1, shape=[128])) 
    h_conv4 = tf.nn.relu(
        tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

    # CONV
    W_conv5 = tf.Variable(
        tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
    b_conv5 = tf.Variable(
        tf.constant(0.1, shape=[128]))
    h_conv5 = tf.nn.relu(
        tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

    # FC
    W_fc1 = tf.Variable(
        tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
    b_fc1 = tf.Variable(
        tf.constant(0.1, shape=[384]))

    h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

    # Dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

    # FC
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
    logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
    y_pred = tf.nn.softmax(logits)

    return y_pred, logits

인풋 아웃풋 데이터, 드롭아웃 확률을 입력받기 위한 플레이스홀더를 정의합니다.

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

CIFAR-10 데이터를 다운로드하고 데이터를 불러옵니다. scalar 형태의 레이블(0~9)을 One-hot Encoding 형태로 변환합니다.

* `tf.squeeze`: 예를 들자면 [1, 2, 1, 3, 1, 1] 배열을 [2, 3] 배열로 만듭니다.

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data()
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis=1)

CNN 그래프를 생성하고, Loss function으로 Cross-entropy를 사용합니다.

* `tf.train.RMSPropOptimizer`: [Root Mean Square Propagation Optimizer](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp).

In [ ]:
y_pred, logits = build_CNN_classifier(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

세션을 열어 테스트를 진행합니다.

In [ ]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        batch = next_batch(128, x_train, y_train_one_hot.eval())
        if i % 100 == 0:
            train_accuracy = accuracy.eval(
                feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
            loss_print = loss.eval(
                feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})

            print("Epoch = %d, Accuracy = %f, Loss = %f" % 
                  (i, train_accuracy, loss_print))
        # Dropout 20%
        sess.run(
            train_step,
            feed_dict={x: batch[0], y: batch[1], keep_prob: 0.8}
        )

    test_accuracy = 0.0  
    for i in range(10):
        test_batch = next_batch(1000, x_test, y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(
            feed_dict={x: test_batch[0], y: test_batch[1], keep_prob: 1.0})
    test_accuracy = test_accuracy / 10;
    print("Total accuracy = %f" % test_accuracy)